In [ ]:
    import os
    from dotenv import load_dotenv
    from langchain_community.document_loaders import PyPDFLoader
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain_chroma import Chroma
    from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
    from langchain.chains import create_retrieval_chain
    from langchain.chains.combine_documents import create_stuff_documents_chain
    from langchain_core.prompts import ChatPromptTemplate

    # Load environment variables (API keys, etc.)
    load_dotenv()

    #   Step 1: Load Multiple PDFs from a Folder
    pdf_folder = "pdfs"  # Change this to your actual folder path

    # Get all PDF files from the folder dynamically
    pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.endswith(".pdf")]

    # Load and merge all PDFs
    docs = []
    for pdf in pdf_files:
        loader = PyPDFLoader(pdf)
        docs.extend(loader.load())  # Append loaded content from each PDF

    print(f" Loaded {len(pdf_files)} PDFs, Total Documents: {len(docs)}")

    #   Step 2: Split text into smaller chunks for better processing
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
    split_docs = text_splitter.split_documents(docs)

    print(f" Split into {len(split_docs)} document chunks.")

    #   Step 3: Initialize the Gemini Embedding Model
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

    #   Step 4: Store Documents in ChromaDB Vector Store
    vectorstore = Chroma.from_documents(documents=split_docs, embedding=embeddings)

    #   Step 5: Create a Retriever for Similarity Search
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

    #   Step 6: Initialize Gemini LLM for Answer Generation
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.3, max_tokens=500)

    #   Step 7: Define the System Prompt
    system_prompt = (
        "You are an assistant for question-answering tasks. "
        "Use the following retrieved context to answer the question. "
        "If you don't know the answer, say you don't know. "
        "Use three sentences maximum and keep the answer concise."
        "\n\n{context}"
    )

    #   Step 8: Create the Prompt Template
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}"),
    ])

    #   Step 9: Create Retrieval & Response Chains
    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)



✅ Loaded 1 PDFs, Total Documents: 13
✅ Split into 43 document chunks.


In [ ]:
#   Step 10: Get Dynamic User Input and Generate Answer
while True:
    user_query = input("\n❓ Enter your question (or type 'exit' to quit): ")
    if user_query.lower() == "exit":
        print("👋 Exiting... Goodbye!")
        break

    # Run the query through the RAG pipeline
    response = rag_chain.invoke({"input": user_query})

    # Display the result
    print("\n💡 Answer:", response["answer"])


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..



💡 Answer: The brain receives sensory input, processes it in specialized areas, and makes decisions.  These decisions are then relayed to motor areas controlling voluntary muscles.  Involuntary actions are controlled by the mid-brain and hind-brain.
👋 Exiting... Goodbye!


In [ ]:
user_query = input("\n Enter your question? ")

    # Run the query through the RAG pipeline
response = rag_chain.invoke({"input": user_query})

    # Display the result
print("\n💡 Answer:", response["answer"])



💡 Answer: The brain has three major parts: the fore-brain, mid-brain, and hind-brain. The fore-brain is responsible for thinking and processing sensory information.  The mid-brain and hind-brain control many involuntary actions.
